In [1]:
import ee
from RadGEEToolbox import LandsatCollection

In [2]:
# Attempt to initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Initialization failed, attempting authentication...")
    try:
        ee.Authenticate()
        ee.Initialize()
        print("Authentication and initialization successful.")
    except Exception as auth_error:
        print("Authentication failed. Error details:", auth_error)

Earth Engine initialized successfully.


**`RadGEEToolbox` has many available spectral index calculations for both Landsat and Sentinel-2 data**

Such indices follow, but are not limited to: NDWI (water), NDVI (vegetation), NDTI (water turbidity), relative chlorophyll-a concentrations, 
and other indices for mapping evaporites (established by the author of this package - see Radwin & Bowen, 2021). Additionally, Land Surface 
Temperature (LST) can be easily calculcated for Landsat imagery.

**In addition to multispectral indices, there are easy ways to temporally reduce image collections to mean, median, min, or max images**

**In the below cell are examples of how to quickly process an entire image collection to the available spectral indices, using the available attribute functions**

In [ ]:
# First we need an image collection
col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38)

#Then it is as easy as calling the desired attribute for multispectral indices
NDWI = col.ndwi
NDVI = col.ndvi
NDTI = col.turbidity
halite = col.halite
gypsum = col.gypsum
chlorophyll = col.chlorophyll
surface_temperature = col.LST

# Print the image information for the last image in the NDWI and surface temperature collections
print(NDWI.image_grab(-1).getInfo())
print(surface_temperature.image_grab(-1).select('LST').getInfo())

{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7801, 7911], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 299985, 0, -30, 4583415]}], 'properties': {'DATUM': 'WGS84', 'REFLECTANCE_ADD_BAND_3': -0.2, 'REFLECTANCE_ADD_BAND_4': -0.2, 'REFLECTANCE_ADD_BAND_5': -0.2, 'REFLECTANCE_ADD_BAND_6': -0.2, 'REFLECTANCE_ADD_BAND_7': -0.2, 'REFLECTIVE_SAMPLES': 7801, 'GROUND_CONTROL_POINTS_VERSION': 5, 'SUN_AZIMUTH': 127.82804601, 'UTM_ZONE': 12, 'DATE_ACQUIRED': '2023-06-29', 'ELLIPSOID': 'WGS84', 'DATA_SOURCE_PRESSURE': 'Calculated', 'LANDSAT_PRODUCT_ID': 'LC09_L2SP_038032_20230629_20230703_02_T1', 'STATION_ID': 'LGN', 'TEMPERATURE_ADD_BAND_ST_B10': 149, 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT', 'REFLECTANCE_MULT_BAND_7': 2.75e-05, 'REFLECTANCE_MULT_BAND_6': 2.75e-05, 'L1_PROCESSING_LEVEL': 'L1TP', 'PROCESSING_SOFTWARE_VERSION': 'LPGS_16.3.0', 'L1_DATE_PRODUCT_GENERATED': '2023-06-29T22:49:40Z', 'ORIENTATION':

**It is also very easy to perform temporal reductions on an entire collection, as shown below**

In [ ]:
# Temporal reductions of image collections
mean_img = col.mean
min_img = col.min
median_img = col.median
max_img = col.max

#Temporal reductions can be applied to multispectral indices as well
mean_chlorophyll = chlorophyll.mean

**The above approach calculates spectral indices but does not perform binary threshold masking / classification**

**If you wish to perform binary thresholding, there are alternative functions for calculating spectral indices and masking based on a floating point threshold value, where values below the threshold are masked as nodata pixels -- as shown below**


In [9]:
#Using methods to calculate multispectral indices

NDWI = col.ndwi_collection(threshold=0)
NDVI = col.ndvi_collection(threshold=0.3)
turbidity = col.turbidity_collection(threshold=0)

#If you will be using Landsat 5 and Landsat 8 or 9 imagery together in a collection, you can specify different thresholds 
# for TM vs OLI sensors where ng_threshold means "next generation" threshold for newer sensors
NDWI = col.ndwi_collection(threshold=0, ng_threshold=0.2)

print(NDWI)
print(NDVI)
print(turbidity)

print(NDWI.collection.getInfo())
print(NDVI.collection.getInfo())
print(turbidity.collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 302085, 0, -30, 4583415]}], 'properties': {'threshold': 0.2, 'DATUM': 'WGS84', 'REFLECTANCE_ADD_BAND_3': -0.2, 'REFLECTANCE_ADD_BAND_4': -0.2, 'REFLECTANCE_ADD_BAND_5': -0.2, 'REFLECTANCE_ADD_BAND_6': -0.2, 'REFLECTANCE_ADD_BAND_7': -0.2, 'REFLECTIVE_SAMPLES': 7801, 'GROUND_CONTROL_POINTS_VERSION': 5, 'SUN_AZIMUTH': 130.95887584, 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION': 'TIRS', 'UTM_ZONE': 12, 'DATE_ACQUIRED': '2023-06-05', 'ELLIPSOID': 'WGS84', 'DATA_SOURCE_PRESSURE': 'Calculated', 'LANDSAT_PRODUCT_ID': 'LC08_L2SP_038032_20230605_20230613_02_T1', 'STATION_ID': 'LGN', 'TEMPERATURE_ADD_BAND_ST_B10': 149, 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT', 'REFLECTANCE_MULT_BAND_7': 2.75e-05, 'REFLECTANCE_MULT_BAND_6': 2.75e-05, 'L1_PROCESSING_LEVEL'

**By directly calling static method functions, `RadGEEToolbox` provides the freedom to use almost any of the core functions, including spectral index calculations, on GEE objects**

In [5]:
#Example using static method function to calculate NDWI for all images in an ee.ImageCollection object
#Note the need for lambda to map functions with arguments other than the input image
NDWI = col.collection.map(lambda image: LandsatCollection.landsat_ndwi_fn(image, threshold=0))
print(NDWI.getInfo())



{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'ndwi', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'dimensions': [7801, 7921], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 302085, 0, -30, 4583415]}], 'properties': {'DATUM': 'WGS84', 'REFLECTANCE_ADD_BAND_3': -0.2, 'REFLECTANCE_ADD_BAND_4': -0.2, 'REFLECTANCE_ADD_BAND_5': -0.2, 'REFLECTANCE_ADD_BAND_6': -0.2, 'REFLECTANCE_ADD_BAND_7': -0.2, 'REFLECTIVE_SAMPLES': 7801, 'GROUND_CONTROL_POINTS_VERSION': 5, 'SUN_AZIMUTH': 130.95887584, 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION': 'TIRS', 'UTM_ZONE': 12, 'DATE_ACQUIRED': '2023-06-05', 'ELLIPSOID': 'WGS84', 'DATA_SOURCE_PRESSURE': 'Calculated', 'LANDSAT_PRODUCT_ID': 'LC08_L2SP_038032_20230605_20230613_02_T1', 'STATION_ID': 'LGN', 'TEMPERATURE_ADD_BAND_ST_B10': 149, 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT', 'REFLECTANCE_MULT_BAND_7': 2.75e-05, 'REFLECTANCE_MULT_BAND_6': 2.75e-05, 'L1_PROCESSING_LEVEL': 'L1TP', 'PROCESS

### *Please refer to the [RadGEEToolbox documentation](https://radgeetoolbox.readthedocs.io/en/latest/) for more information and a comprehensive list of available functionality*